In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

df = pd.read_excel('result.xlsx')


In [4]:
# Loại bỏ cột không liên quan
df = df.drop(['Dấu thời gian', 'Tên Sinh viên', 'Giới tính'], axis=1)


In [5]:
# Chuyển đổi cột "Ngành học" thành one-hot encoding
df = pd.get_dummies(df, columns=['Ngành học'])


In [6]:
print(df)


     1. Tôi có tính tự lập  2. Tôi suy nghĩ thực tế  \
0                        3                        3   
1                        3                        4   
2                        3                        4   
3                        2                        2   
4                        2                        2   
..                     ...                      ...   
353                      4                        5   
354                      4                        5   
355                      1                        1   
356                      5                        5   
357                      5                        5   

     3. Tôi là người thích nghi với môi trường mới  \
0                                                4   
1                                                3   
2                                                5   
3                                                3   
4                                                2   
..             

In [7]:
# Xác định tất cả các cột đại diện cho ngành học
nganh_hoc_cols = [col for col in df.columns if col.startswith('Ngành học_')]


In [8]:
# Tạo cột mục tiêu "Ngành học_Dự đoán" 
df['Ngành học_Dự đoán'] = 0
for i, row in df.iterrows():
    for col in df.columns:
        if col.startswith('Ngành học_') and row[col] == 1:
            df.loc[i, 'Ngành học_Dự đoán'] = col
            break


C:\Users\LamPC\AppData\Local\Temp\ipykernel_17668\2465886777.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Ngành học_Sư phạm hóa học' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Ngành học_Dự đoán'] = col


In [9]:
df['Ngành học_Dự đoán'].value_counts()


Ngành học_Dự đoán
Ngành học_Công nghệ thông tin              64
Ngành học_Giáo dục tiểu học                43
Ngành học_Ngôn ngữ anh                     39
Ngành học_Luật                             38
Ngành học_Kế toán                          28
Ngành học_Sư phạm hóa học                  26
Ngành học_Quản trị kinh doanh              18
Ngành học_Tâm lý học giáo dục              17
Ngành học_Sư phạm tin học                  13
Ngành học_Kỹ thuật điện                    13
Ngành học_Sư phạm tiếng anh                 9
Ngành học_Công tác xã hội                   8
Ngành học_Tài chính - Ngân hàng             5
Ngành học_Quản lý nhà nước                  4
Ngành học_Sư phạm toán học                  3
Ngành học_Quản lý giáo dục                  3
Ngành học_Nông học                          3
Ngành học_Toán học                          3
Ngành học_Công nghệ kỹ thuật hóa học        2
Ngành học_Văn học                           2
Ngành học_Quản trị khách sạn                2
Ngành học_Kinh t

In [10]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Giả sử bạn có một tập dữ liệu với hai đặc trưng
data = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])

# Tạo một đối tượng StandardScaler
scaler = StandardScaler()

# Huấn luyện scaler và chuẩn hóa dữ liệu huấn luyện
scaled_data = scaler.fit_transform(data)

print("Dữ liệu ban đầu:")
print(data)
print("\nDữ liệu sau khi chuẩn hóa:")
print(scaled_data)

# Bạn có thể sử dụng scaler để chuẩn hóa dữ liệu mới
new_data = np.array([[9, 10]])
scaled_new_data = scaler.transform(new_data)
print("\nDữ liệu mới sau khi chuẩn hóa:")
print(scaled_new_data)


Dữ liệu ban đầu:
[[1 2]
 [3 4]
 [5 6]
 [7 8]]

Dữ liệu sau khi chuẩn hóa:
[[-1.34164079 -1.34164079]
 [-0.4472136  -0.4472136 ]
 [ 0.4472136   0.4472136 ]
 [ 1.34164079  1.34164079]]

Dữ liệu mới sau khi chuẩn hóa:
[[2.23606798 2.23606798]]


In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

# Xử lý mất cân bằng dữ liệu bằng oversampling
def balance_data(df, target_column):
    max_size = df[target_column].value_counts().max()
    lst = [df]
    for class_index, group in df.groupby(target_column):
        lst.append(group.sample(max_size-len(group), replace=True))
    df_balanced = pd.concat(lst)
    return df_balanced

df_balanced = balance_data(df, 'Ngành học_Dự đoán')

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X = df_balanced.drop('Ngành học_Dự đoán', axis=1).iloc[:,:54]  # Lấy 54 cột đầu tiên
y = df_balanced['Ngành học_Dự đoán']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Khởi tạo các mô hình
logistic_model = LogisticRegression(max_iter=1000)
rf_model = RandomForestClassifier()
gb_model = GradientBoostingClassifier()

# Xây dựng danh sách siêu tham số muốn thử nghiệm
param_grid_lr = {'C': [0.1, 1, 10, 100]}
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
param_grid_gb = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5]}

# Khởi tạo GridSearchCV cho từng mô hình
grid_search_lr = GridSearchCV(logistic_model, param_grid_lr, cv=5, n_jobs=-1)
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, n_jobs=-1)
grid_search_gb = GridSearchCV(gb_model, param_grid_gb, cv=5, n_jobs=-1)

# Huấn luyện mô hình với GridSearchCV
grid_search_lr.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)
grid_search_gb.fit(X_train, y_train)

# Lấy mô hình tốt nhất
best_model_lr = grid_search_lr.best_estimator_
best_model_rf = grid_search_rf.best_estimator_
best_model_gb = grid_search_gb.best_estimator_

# Dự đoán với mô hình tốt nhất
y_pred_lr = best_model_lr.predict(X_test)
y_pred_rf = best_model_rf.predict(X_test)
y_pred_gb = best_model_gb.predict(X_test)

# Đánh giá hiệu suất
print("Logistic Regression:")
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))

print("Random Forest:")
print(classification_report(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))

print("Gradient Boosting:")
print(classification_report(y_test, y_pred_gb))
print(confusion_matrix(y_test, y_pred_gb))


Logistic Regression:
                                         precision    recall  f1-score   support

   Ngành học_Công nghệ kỹ thuật hóa học       1.00      1.00      1.00        13
  Ngành học_Công nghệ kỹ thuật xây dựng       1.00      1.00      1.00        11
          Ngành học_Công nghệ thông tin       0.25      0.14      0.18        14
              Ngành học_Công tác xã hội       0.93      1.00      0.97        14
           Ngành học_Giáo dục chính trị       1.00      1.00      1.00        14
             Ngành học_Giáo dục mầm non       1.00      1.00      1.00         9
            Ngành học_Giáo dục tiểu học       0.64      0.64      0.64        11
                      Ngành học_Kinh tế       1.00      1.00      1.00        10
                      Ngành học_Kế toán       0.87      0.87      0.87        15
            Ngành học_Kỹ thuật phần mềm       1.00      1.00      1.00        12
                Ngành học_Kỹ thuật điện       0.90      1.00      0.95         9
Ngành 

In [12]:
import numpy as np
import pandas as pd

# Tạo DataFrame mới chứa dữ liệu câu hỏi của người dùng
user_data = pd.DataFrame([
    [4, 5, 3, 2, 2, 4, 3, 3, 5, 2, 3, 3, 4, 4, 4, 2, 4, 3, 3, 5, 5, 2, 3, 3, 3, 3, 4, 5, 4, 5, 4, 5, 5, 4, 5, 4, 3, 5, 3, 3, 4, 3, 1, 5, 5, 3, 5, 5, 4, 2, 2, 5, 4, 3]
])
# Điền giá trị mặc định hoặc giả vào các cột còn thiếu
# Điền giá trị mặc định là 0 vào các cột còn thiếu
# user_data_filled = user_data.copy()  # Tạo một bản sao của dữ liệu người dùng
# user_data_filled = user_data_filled.reindex(columns=range(85), fill_value=0)  # Điền giá trị mặc định là 0 vào các cột còn thiếu

# Dự đoán ngành học cho người dùng mới
y_pred_user_lr = best_model_lr.predict(user_data)  # Dự đoán bằng Logistic Regression
y_pred_user_rf = best_model_rf.predict(user_data)  # Dự đoán bằng Random Forest
y_pred_user_gb=best_model_gb.predict(user_data)

# Hiển thị kết quả dự đoán
print(f'Ngành học dự đoán cho người dùng (Logistic Regression): {y_pred_user_lr[0]}')
print(f'Ngành học dự đoán cho người dùng (Random Forest): {y_pred_user_rf[0]}')
print(f'Ngành học dự đoán cho người dùng (Random Forest): {y_pred_user_gb[0]}')



Ngành học dự đoán cho người dùng (Logistic Regression): Ngành học_Công tác xã hội
Ngành học dự đoán cho người dùng (Random Forest): Ngành học_Công nghệ thông tin
Ngành học dự đoán cho người dùng (Random Forest): Ngành học_Công nghệ thông tin


In [13]:
import pickle

# Lưu mô hình tốt nhất (ví dụ với Random Forest)
with open('model/model.pkl', 'wb') as file:
    pickle.dump(best_model_rf, file)
